# 全连接神经网络实际数据分析

在之前的章节中，我们介绍了神经网络的基本概念、以及如何对其进行编程。在这一章节中，我们以上市公司税收不遵从行为的分类为例，用```torch```展示神经网络的内部工作机制，并借此简单介绍```torch```的基本用法。

**学习目标**：
* a
* b
* c

## 目录

<a name='1'></a>
## 1-第三方包导入

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

<a name='2'></a>
## 2-探索性数据分析

我们使用的数据包括2015-2024年中国所有上市公司的财报指标和纳税违规公告。由于税收遵从的公司占了绝大部分，我们对其进行下采样。最终得到的数据集data.csv中包含4155条税收遵从企业和831条不遵从企业的观测记录。

下面我们先使用```pandas```读取数据文件，再展示数据集的前5条观测值，以对数据产生初步认识。

In [18]:
data = pd.read_csv('data.csv') # 读取数据
data.head(5) # 展示数据前5行


,noncompliance,股东类别_SHType,股东总户数(户)_SHNum,户均持股数(股/户)_AvgHS,每股收益(元/股)_BasicEPS,每股经营活动现金流量(元/股)_OpeCFPS,每股现金及现金等价物余额(元/股)_CCEPS,净利润()_NetPrf,营业利润/营业总收入()_OpePrTOR,有形净值债务率(%)_DbTanEquRt,股东权益/负债合计_EquTotLia,资产负债率(%)_DbAstRt,产权比率(%)_DbEquRt,利润总额增长率(%)_TotPrfGrRt,营业收入3年复合增长率(%)_OperaInc3GrRt,股东权益周转率(次)_EquRat,经营现金净流量(元)_NOCF,营业收入增长率(%)_OpeIncmGrRt,每股资本公积金(元/股)_CapSurFdPS,权益乘数(%)_EquMul
0,1.0,0,47637.0,13149.0,0.6100,0.8540,1.8344,3.821158e+08,10.8036,57.8121,188.6212,33.5835,50.5650,-13.7859,11.3971,1.0675,5.349618e+08,8.8828,0.2677,1.5056
1,1.0,1,45243.0,35258.0,0.0100,-1.1303,0.8549,1.805186e+07,-45.4209,334.6187,35.4071,73.4626,276.8262,101.8748,-23.1548,2.2301,-1.802977e+09,5.3417,0.8399,3.7683
2,1.0,0,12732.0,22550.0,1.8600,1.1441,0.7015,5.155666e+08,28.0363,47.1087,297.0299,25.1890,33.6701,23.9927,22.9617,0.9925,3.284657e+08,81.9351,2.6566,1.3367
3,1.0,0,21691.0,10142.0,0.3539,0.0594,0.7108,7.786043e+07,4.0816,138.3406,80.9723,51.7863,107.4101,31.7674,11.1301,2.2031,1.307214e+07,12.8161,1.1694,2.0741
4,1.0,1,35615.0,8514.0,0.1809,0.5599,1.4905,5.484244e+07,10.1237,71.0228,141.8598,41.1106,69.8098,8.9495,8.9858,0.9102,1.697918e+08,-0.8419,0.2838,1.6981


在该数据集中，```noncompliance``` 变量是我们关心的Y变量，```noncompliance = 1```代表该企业在当年存在着税收不遵从行为。

我们按照```noncompliance```变量进行分组，在每组中计算样本均值。可以发现税收不遵从的企业和税收遵从的企业在股权结构、偿债能力、盈利能力等方面都可能存在着不同。

In [ ]:
data.groupby('noncompliance').mean().round(3).T # 按照noncompliance分组计算均值，取3位小数

noncompliance,0.0,1.0
股东类别_SHType,2.630000e-01,2.610000e-01
股东总户数(户)_SHNum,4.715383e+04,5.514588e+04
户均持股数(股/户)_AvgHS,9.596032e+05,2.308614e+06
每股收益(元/股)_BasicEPS,5.400000e-01,3.420000e-01
每股经营活动现金流量(元/股)_OpeCFPS,6.870000e-01,5.890000e-01
每股现金及现金等价物余额(元/股)_CCEPS,2.025000e+00,2.289000e+00
净利润()_NetPrf,1.015824e+09,1.771778e+09
营业利润/营业总收入()_OpePrTOR,2.735000e+00,-3.152000e+00
有形净值债务率(%)_DbTanEquRt,7.832400e+01,2.715530e+02
股东权益/负债合计_EquTotLia,2.397120e+02,1.409650e+02


<a name='3'></a>
## 3-建模分析

我们首先将数据集划分为训练集和测试机